In [33]:
# # 📦 Imports
# from fredapi import Fred
# import pandas as pd
# import os

# # 🔧 Setup
# API_KEY = "YOUR API KEY HERE"
# fred = Fred(api_key=API_KEY)
# start_date = "2000-01-01"
# end_date = "2025-01-01"
# freq = "ME"  # Monthly frequency

# # ✅ Feature mapping: FRED series → custom column names
# features = {
#     "TOTALSL": "layman__consumer_credit",                      # Total consumer loans
#     "TERMCBAUTO48NS": "layman__auto_loan_rate",                # Avg 48-month new auto loan rate
#     "SLOAS": "layman__student_debt_level",                     # Total student loan debt
#     "RRVRUSQ156N": "layman__rental_vacancy_rate",              # Quarterly → ffill to monthly
#     # "CUUR0000SETA03": "layman__fast_food_price_index",         # CPI: Limited Service Meals
#     "CUSR0000SETA02": "layman__fast_food_price_index",     # ✅ CORRECTED
#     "CUSR0000SEEA": "layman__concert_ticket_index",            # CPI: Admission to events
#     "CUSR0000SETG01": "layman__airfare_index",                 # CPI: Airline fares
#     # "CUSR0000SEEB03": "layman__childcare_cost_index",          # CPI: Childcare
#     "CUSR0000SEEB": "layman__education_childcare_index",
#     "CUSR0000SETG": "layman__public_transit_fares",            # CPI: Public transit
# }

# # 🗓️ Create monthly date index and base DataFrame
# date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
# df = pd.DataFrame(index=date_index)

# # 📥 Download FRED data
# for code, alias in features.items():
#     try:
#         series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
#         series = series.resample(freq).ffill()
#         df[alias] = series
#         print(f"✅ Loaded {alias}")
#     except Exception as e:
#         print(f"❌ Failed to load {alias}: {e}")

# # 🧠 Netflix Subscriptions Placeholder
# df["layman__netflix_subs_index"] = pd.NA
# print("🧱 Placeholder added: layman__netflix_subs_index")

# # 🧼 Final formatting
# df.index.name = "date"
# df = df.ffill()

# # 💾 Save to CSV
# os.makedirs("layman_batches", exist_ok=True)
# df.to_csv("layman_batches/Batch_Layman_02.csv")
# print("✅ Saved Batch_Layman_02.csv")

In [49]:
# # 📦 Imports
# from fredapi import Fred
# from pytrends.request import TrendReq
# import pandas as pd
# import os
# from sklearn.linear_model import LinearRegression
# import numpy as np

# # 🔧 Setup
# API_KEY = "YOUR API KEY HERE"
# fred = Fred(api_key=API_KEY)
# start_date = "2000-01-01"
# end_date = "2025-01-01"
# freq = "ME"  # Monthly frequency

# # ✅ Feature mapping: FRED series → custom column names
# features = {
#     "TOTALSL": "layman__consumer_credit",
#     "TERMCBAUTO48NS": "layman__auto_loan_rate",
#     "SLOAS": "layman__student_debt_level",
#     "RRVRUSQ156N": "layman__rental_vacancy_rate",
#     "CUSR0000SETA02": "layman__fast_food_price_index",
#     "CUSR0000SEEA": "layman__concert_ticket_index",
#     "CUSR0000SETG01": "layman__airfare_index",
#     "CUSR0000SEEB": "layman__education_childcare_index",
#     "CUSR0000SETG": "layman__public_transit_fares",
# }

# # 🗓️ Create monthly date index and base DataFrame
# date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
# df = pd.DataFrame(index=date_index)

# # 📥 Download FRED data
# for code, alias in features.items():
#     try:
#         series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
#         series = series.resample(freq).ffill()
#         df[alias] = series
#         print(f"✅ Loaded {alias}")
#     except Exception as e:
#         print(f"❌ Failed to load {alias}: {e}")

# # 🔎 Add Google Trends as proxy for Netflix subscription interest
# try:
#     pytrends = TrendReq()
#     pytrends.build_payload(kw_list=["Netflix subscription"], timeframe="2004-01-01 2025-01-01")
#     trends = pytrends.interest_over_time()
#     if not trends.empty:
#         netflix_trend = trends["Netflix subscription"].resample("ME").mean()
#         netflix_trend = netflix_trend.reindex(df.index).ffill()
#         df["layman__netflix_subs_index"] = netflix_trend
#         print("✅ Loaded layman__netflix_subs_index from Google Trends")
#     else:
#         df["layman__netflix_subs_index"] = pd.NA
#         print("⚠️ Google Trends returned no data, placeholder used")
# except Exception as e:
#     df["layman__netflix_subs_index"] = pd.NA
#     print(f"⚠️ Failed to fetch Google Trends for Netflix: {e}")

# # 🧽 Fill edge missing values in auto loan rate
# df["layman__auto_loan_rate"].fillna(method="bfill", inplace=True)  # use next value since it's the first row

# # Multivariate Linear Regression for Filling layman__student_debt_level where missing
# notna_mask = df["layman__student_debt_level"].notna()
# X_train = df.loc[notna_mask, ["layman__consumer_credit"]].fillna(method="bfill")
# y_train = df.loc[notna_mask, "layman__student_debt_level"]

# model = LinearRegression().fit(X_train, y_train)

# missing_mask = df["layman__student_debt_level"].isna()
# X_missing = df.loc[missing_mask, ["layman__consumer_credit"]].fillna(method="bfill")

# predicted = model.predict(X_missing)
# df.loc[missing_mask, "layman__student_debt_level"] = predicted

# print("✅ Filled layman__student_debt_level using consumer credit as a predictor.")


# # 🧼 Final formatting
# df.index.name = "date"
# df = df.ffill()

# # 💾 Save to CSV
# os.makedirs("layman_batches", exist_ok=True)
# df.to_csv("layman_batches/Batch_Layman_02.csv")
# print("✅ Saved Batch_Layman_02.csv")

In [59]:
# 📦 Imports
from fredapi import Fred
from pytrends.request import TrendReq
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
import numpy as np

# 🔧 Setup
API_KEY = "YOUR API KEY HERE"
fred = Fred(api_key=API_KEY)
start_date = "2000-01-01"
end_date = "2025-01-01"
freq = "ME"  # Monthly frequency

# ✅ Feature mapping: FRED series → custom column names
features = {
    "TOTALSL": "layman__consumer_credit",
    "TERMCBAUTO48NS": "layman__auto_loan_rate",
    "SLOAS": "layman__student_debt_level",
    "RRVRUSQ156N": "layman__rental_vacancy_rate",
    "CUSR0000SETA02": "layman__fast_food_price_index",
    "CUSR0000SEEA": "layman__concert_ticket_index",
    "CUSR0000SETG01": "layman__airfare_index",
    "CUSR0000SEEB": "layman__education_childcare_index",
    "CUSR0000SETG": "layman__public_transit_fares",
}

# 🗓️ Create monthly date index and base DataFrame
date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
df = pd.DataFrame(index=date_index)

# 📥 Download FRED data
for code, alias in features.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        series = series.resample(freq).ffill()
        df[alias] = series
        print(f"✅ Loaded {alias}")
    except Exception as e:
        print(f"❌ Failed to load {alias}: {e}")

# 🔎 Add Google Trends as proxy for Netflix subscription interest
# try:
#     pytrends = TrendReq()
#     pytrends.build_payload(kw_list=["Netflix subscription"], timeframe="2004-01-01 2025-01-01")
#     trends = pytrends.interest_over_time()
#     if not trends.empty:
#         netflix_trend = trends["Netflix subscription"].resample("ME").mean()
#         netflix_trend = netflix_trend.reindex(df.index).ffill()
#         df["layman__netflix_subs_index"] = netflix_trend
#         print("✅ Loaded layman__netflix_subs_index from Google Trends")
#     else:
#         df["layman__netflix_subs_index"] = pd.NA
#         print("⚠️ Google Trends returned no data, placeholder used")
# except Exception as e:
#     df["layman__netflix_subs_index"] = pd.NA
#     print(f"⚠️ Failed to fetch Google Trends for Netflix: {e}")
# 🎯 Replace Netflix trend with real CPI for video streaming

# CPI-U: Video and Audio Subscription Services
# Measures the average monthly price change for video and audio streaming services 
# (e.g., Netflix, Hulu, Spotify) paid by urban U.S. consumers.

try:
    series = fred.get_series("CUSR0000SERA02", observation_start=start_date, observation_end=end_date)
    series = series.resample(freq).ffill()
    df["layman__streaming_cost_index"] = series
    print("✅ Loaded layman__streaming_cost_index from FRED (video & audio subscription services)")
except Exception as e:
    df["layman__streaming_cost_index"] = pd.NA
    print(f"❌ Failed to load streaming cost index: {e}")

# # 🧽 Fill edge missing values in auto loan rate
# df["layman__auto_loan_rate"].fillna(method="bfill", inplace=True)  # use next value since it's the first row

# # 🤖 Fill missing student debt levels using consumer credit as a predictor
# notna_mask = df["layman__student_debt_level"].notna()
# X_train = df.loc[notna_mask, ["layman__consumer_credit"]].fillna(method="bfill")
# y_train = df.loc[notna_mask, "layman__student_debt_level"]

# model = LinearRegression().fit(X_train, y_train)

# missing_mask = df["layman__student_debt_level"].isna()
# # X_missing = df.loc[missing_mask, ["layman__consumer_credit"]].fillna(method="bfill")
# df["layman__auto_loan_rate"] = df["layman__auto_loan_rate"].bfill()

# predicted = model.predict(X_missing)
# df.loc[missing_mask, "layman__student_debt_level"] = predicted

# print("✅ Filled layman__student_debt_level using consumer credit as a predictor.")

# 🧽 Fill edge missing values in auto loan rate
df["layman__auto_loan_rate"] = df["layman__auto_loan_rate"].bfill()

# 🤖 Fill missing student debt levels using consumer credit as a predictor
notna_mask = df["layman__student_debt_level"].notna()
X_train = df.loc[notna_mask, ["layman__consumer_credit"]].bfill()
y_train = df.loc[notna_mask, "layman__student_debt_level"]

model = LinearRegression().fit(X_train, y_train)

missing_mask = df["layman__student_debt_level"].isna()
X_missing = df.loc[missing_mask, ["layman__consumer_credit"]].bfill()

predicted = model.predict(X_missing)
df.loc[missing_mask, "layman__student_debt_level"] = predicted

print("✅ Filled layman__student_debt_level using consumer credit as a predictor.")

# 🧼 Final formatting
df.index.name = "date"
df = df.ffill()

# 💾 Save to CSV
os.makedirs("layman_batches", exist_ok=True)
df.to_csv("layman_batches/Batch_Layman_02.csv")
print("✅ Saved Batch_Layman_02.csv")

✅ Loaded layman__consumer_credit
✅ Loaded layman__auto_loan_rate
✅ Loaded layman__student_debt_level
✅ Loaded layman__rental_vacancy_rate
✅ Loaded layman__fast_food_price_index
✅ Loaded layman__concert_ticket_index
✅ Loaded layman__airfare_index
✅ Loaded layman__education_childcare_index
✅ Loaded layman__public_transit_fares
✅ Loaded layman__streaming_cost_index from FRED (video & audio subscription services)
✅ Filled layman__student_debt_level using consumer credit as a predictor.
✅ Saved Batch_Layman_02.csv
